## The code below,

1. Matches the Prolific ID to the Participant ID of the survey users & extracts the duration of time taken by the users to complete the survey
2. Calculates the Attention Score of the survey users with respect to the stimulus video
3. Appends the newUsers to the participant_log of the existing users
4. Extracts the number of responses 

We find this information through the Survey User Response form exported from Qualtrics

In [1]:
import os
import re
import pandas as pd
import datetime

In [2]:
studyDataResponseLogDirectory = '/Users/sukruthgl/Desktop/Spring/CS 5999 - MEng Project/StudyData/pilots/Bad Robots Empathy_May 20, 2023_14.26.csv'
participantLogDirectory = '/Users/sukruthgl/Desktop/Spring/CS 5999 - MEng Project/StudyData/pilots/participant_log.xlsx'
prolificId_match_directory = '/Users/sukruthgl/Desktop/Spring/CS 5999 - MEng Project/StudyData/pilots/'

In [3]:
studyDataResponseLog_df = pd.read_csv(studyDataResponseLogDirectory)
participant_log = pd.read_excel(participantLogDirectory)

#### 1. Match Prolific ID to the Participant & extract survey time

In [4]:
# Here, we create a dictionary of - {Prolific ID: (randomID, Duration (in seconds))}
# It is then converted into a list of required format which is used to create the
# prolificID_matches_df dataframe

prolificID_dict = {}
participantIDPattern = r'\d{4}'

for index, row in studyDataResponseLog_df.iterrows():
    prolificID_dict[row['randomID']] = row['Q78'], row['Duration (in seconds)']

prolificID_dict = {key: value for key, value in prolificID_dict.items() if re.match(participantIDPattern, key)}
prolificID_matches_lst = [(key, value1, str(datetime.timedelta(seconds=int(value2)))) for key, (value1, value2) in prolificID_dict.items()]


# Creating a df of the Participant ID and their corresponding Prolific User ID
prolificID_matches_df = pd.DataFrame(prolificID_matches_lst, columns = ['Participant', 'Prolific ID', 'Survey Duration'])

prolificID_matches_df

Participant               Prolific ID Survey Duration
0         8006  602e9c46d37c26a6f0ec65ad         0:33:17
1         3260  5faa741b7e564e3971432a66         0:40:56
2         7444  5f2ef7740f87702b99055397         0:33:07
3         3518  6162226f379aa5c552808895         1:06:47
4         9214  601903a3842bcc20df15b296         0:40:17
5         8417  5b8e8feeee5b0500010f16d0         0:27:14
6         6110  6163fe9cd62993d57a578f0c         0:37:33
7         1889  5f4894201c384e3607e91d57         0:45:49
8         1357  6168174598124da7e933f76d         1:02:11
9         6953  5d40a12f4994c40001e4b80c         0:37:54
10        8586  5ddd2b7887c1fdc8d0375bd6         0:23:48
11        5177  5bc1f0157870b900012dcb8a         0:27:04
12        5531  60ae642a030910da26e53328         0:45:51
13        1941  60d090dc7a9fcd3436d75960         0:35:56
14        4725  5fd24dc25191560a0dbe4c66         0:33:46
15        2039  60009e21e71d6d170204369d         0:29:03
16        6686  615da788698e89290fe145e3         0:26:19
17        6253  5fd2159a30a24428526f9c69         0:44:45
18        6566  6029795d5873ea8d25c1bfc9         0:32:47
19        1980                       NaN         0:01:35
20        7440  5db9b910001ffa0188426dca         0:20:59
21        3147  60e6ca61f1fbd1edc317a63e         1:14:35
22        2584  5f6214a59413141f7cb2508c         0:52:56
23        2946  6124c375109ed10432fa2114         0:05:41
24        5077                       NaN         0:00:44
25        2524  6168174598124da7e933f76d         0:23:00
26        3499  5d595c1acf514a00177872ed         0:20:22
27        2987  5ed5c67a751f9814f8ffd344         0:24:22
28        2088  6117bdb3ee7e7ca452e3199d         0:26:24
29        6618  616dc0a86b3ccbe7d735f712         0:33:43
30        1480  60f97c4d9f14764c11aefeea         0:43:59
31        1525  60476cc0bb649624b9661b87         0:39:52
32        8205  615da788698e89290fe145e3         0:09:08
33        8296  60e5a4a93d30a9f7f695a9b7         0:23:38
34        2903  61309cc42d77788e1605c645         0:18:40
35        3947  5f346965f972db371654b614         0:30:42
36        3503  60f046be4df6e302b34526bf         0:31:02
37        7811  6031a3ce9c4b051348c05c4b         0:35:26
38        1499  5e15bb77e78021b9d45d8cf4         0:33:12
39        8845  616711b01b37853903a099e0         0:42:05
40        2101  615e222d390962ba321edde6         0:45:14
41        3235  60ca98d006f5840320e8049e         0:26:31
42        2818  5e9c3ff1cf719d151612648f         1:06:55
43        1230  615ad6db4cd0092a52d09ea7         0:02:55
44        3885                       NaN         0:02:02

#### 2. Calculate Attention Score of the Survey Users

In [5]:
# Identifies all the columns of stimulusVideo required to calculate the Attention Score
attentionCheckerPattern = r'Please indicate your agreement with the following statement.*'
attentionCheckerResponseColumns = []

for column in studyDataResponseLog_df.columns:
    if re.match(attentionCheckerPattern, studyDataResponseLog_df[column].iloc[0]):
        attentionCheckerResponseColumns.append(column)
attentionCheckerResponseColumns.append('randomID')
print(attentionCheckerResponseColumns)

['Q163#1_1', 'Q754#1_1', 'Q761#1_1', 'Q768#1_1', 'Q775#1_1', 'Q782#1_1', 'Q789#1_1', 'Q796#1_1', 'Q803#1_1', 'Q810#1_1', 'Q817#1_1', 'Q824#1_1', 'Q831#1_1', 'Q838#1_1', 'Q845#1_1', 'Q852#1_1', 'Q859#1_1', 'Q866#1_1', 'Q873#1_1', 'Q880#1_1', 'Q887#1_1', 'Q894#1_1', 'Q901#1_1', 'Q908#1_1', 'Q915#1_1', 'Q922#1_1', 'Q929#1_1', 'Q936#1_1', 'Q943#1_1', 'Q950#1_1', 'randomID']


In [6]:
# We identify the context response columns 
# (i.e context reponse columns of the corresponding stimulusVideos respectively)
# which is used for labelling the class of the stimulusVideos & calculating the attention scores
# and return those column names

def contextResponseColumns(columns):
    contextResponseColumnPattern = r"Q\d+#\d_\d"
    
    columns = [col for col in columns if re.match(contextResponseColumnPattern, col)]
    return columns

In [7]:
# Here we label the class the stimulusVideos to its respective context response columns
# This dictionary is used in calculating the attention score.

context_dict = {}

context = {1 : 'FAIL', 
           2: 'CONTROL'}

for contextQuestionID in contextResponseColumns(attentionCheckerResponseColumns):
    contextKey = int(input(f'Enter the context for the video: {contextQuestionID} : '))
    context_dict[contextQuestionID] = context[contextKey]

Enter the context for the video: Q163#1_1 : 2
Enter the context for the video: Q754#1_1 : 2
Enter the context for the video: Q761#1_1 : 2
Enter the context for the video: Q768#1_1 : 2
Enter the context for the video: Q775#1_1 : 2
Enter the context for the video: Q782#1_1 : 2
Enter the context for the video: Q789#1_1 : 2
Enter the context for the video: Q796#1_1 : 2
Enter the context for the video: Q803#1_1 : 2
Enter the context for the video: Q810#1_1 : 2
Enter the context for the video: Q817#1_1 : 1
Enter the context for the video: Q824#1_1 : 1
Enter the context for the video: Q831#1_1 : 1
Enter the context for the video: Q838#1_1 : 1
Enter the context for the video: Q845#1_1 : 1
Enter the context for the video: Q852#1_1 : 1
Enter the context for the video: Q859#1_1 : 1
Enter the context for the video: Q866#1_1 : 1
Enter the context for the video: Q873#1_1 : 1
Enter the context for the video: Q880#1_1 : 1
Enter the context for the video: Q887#1_1 : 1
Enter the context for the video: Q

In [8]:
# context_dict

In [9]:
# Here, we identify the valid prolific IDs 
# We drop all the duplicate entries (this way, if a surveyer has attempted to perform the survey twice, we identify only the first survey)
valid_prolific_ids = list(prolificID_matches_df["Prolific ID"])

studyDataResponseLog_df = studyDataResponseLog_df.drop_duplicates(subset='Q78', keep='first')
# studyDataResponseLog_df[['Q78', 'randomID']]
# len(studyDataResponseLog_df)

In [10]:
# Here, we create the attention_df 
# This consists of all the context response columns of the corresponding stimulusVideos
# We drop all the entries in the prolific ID column (identified by the header 'Q78') that are != 24
# This way we remove all the pilot_test entries

attention_df = studyDataResponseLog_df[attentionCheckerResponseColumns].drop(studyDataResponseLog_df[studyDataResponseLog_df['Q78'].apply(lambda x: len(str(x)) != 24)].index)
attention_df
# len(attention_df)

Q163#1_1                   Q754#1_1  \
10             Strongly agree             Somewhat agree   
11             Strongly agree                      Agree   
12             Strongly agree             Strongly agree   
13             Strongly agree             Strongly agree   
21             Somewhat agree             Somewhat agree   
22             Strongly agree             Strongly agree   
23                      Agree                      Agree   
24                      Agree             Somewhat agree   
25             Somewhat agree  Neither agree or disagree   
26                      Agree                      Agree   
27             Strongly agree             Strongly agree   
28             Somewhat agree             Somewhat agree   
29             Strongly agree             Strongly agree   
30                      Agree                      Agree   
31             Strongly agree             Strongly agree   
32             Strongly agree                      Agree   
33             Strongly agree             Strongly agree   
34             Strongly agree  Neither agree or disagree   
35             Strongly agree                      Agree   
37             Strongly agree             Strongly agree   
38             Strongly agree                      Agree   
39             Strongly agree                      Agree   
40                        NaN                        NaN   
43                      Agree                      Agree   
44             Strongly agree             Strongly agree   
45  Neither agree or disagree                   Disagree   
46             Strongly agree             Strongly agree   
47             Strongly agree                      Agree   
48             Strongly agree             Strongly agree   
50                        NaN                        NaN   
51             Strongly agree             Strongly agree   
52             Strongly agree             Strongly agree   
53                      Agree                      Agree   
54  Neither agree or disagree  Neither agree or disagree   
55             Strongly agree             Somewhat agree   
56             Strongly agree             Strongly agree   
57                      Agree                      Agree   
58                      Agree                   Disagree   
59             Strongly agree             Strongly agree   
60                        NaN                        NaN   

                     Q761#1_1                   Q768#1_1  \
10             Somewhat agree                   Disagree   
11                      Agree                      Agree   
12             Strongly agree             Strongly agree   
13                      Agree  Neither agree or disagree   
21             Somewhat agree  Neither agree or disagree   
22             Strongly agree             Strongly agree   
23                      Agree             Strongly agree   
24  Neither agree or disagree  Neither agree or disagree   
25                   Disagree                   Disagree   
26                      Agree             Somewhat agree   
27             Strongly agree             Strongly agree   
28             Somewhat agree             Somewhat agree   
29             Strongly agree                      Agree   
30             Strongly agree          Somewhat disagree   
31             Strongly agree             Strongly agree   
32                      Agree                      Agree   
33             Strongly agree             Strongly agree   
34                      Agree                      Agree   
35             Strongly agree                      Agree   
37             Strongly agree             Strongly agree   
38             Strongly agree             Strongly agree   
39             Strongly agree             Strongly agree   
40                        NaN                        NaN   
43                      Agree                      Agree   
44             Strongly agree             Strongly agree   


In [11]:
# Here, we define the attention_dict used in attention_score calculation

attention_dict = {'Strongly disagree': 0,
                 'Disagree': 1,
                 'Somewhat disagree': 2,
                 'Neither agree or disagree': 3,
                 'Somewhat agree': 4,
                 'Agree': 5,
                 'Strongly agree': 6}

In [12]:
# Here we calculate the attention_score for each surveyer

attention_scores = []

for index, row in attention_df.filter(items = contextResponseColumns(attention_df.columns), axis = 1).iterrows():
    attentionScore = 0
    for columnName, columnValue in row.items():
        try:
#             print(f'{columnName}: {context_dict[columnName]} Video, Response: {columnValue}, Response Value = {attention_dict[columnValue]}')
            if(context_dict[columnName] == 'CONTROL'):
                if (attention_dict[columnValue] > 3):
                    attentionScore += 1
                elif(attention_dict[columnValue] < 3):
                    attentionScore -= 1
                else:
                    attentionScore += 0
            elif(context_dict[columnName] == 'FAIL'):
                if (attention_dict[columnValue] < 3):
                    attentionScore += 1
                elif(attention_dict[columnValue] > 3):
                    attentionScore -= 1
                else:
                    attentionScore += 0
        except KeyError:
            # Any survey entry where there are missing responses - penalise them heavily - to indicate not to be considered for study
            attentionScore += -10
        except AttributeError:
            attentionScore += -10
#         print(f'{columnName}: {context_dict[columnName]} Video, Response: {columnValue}, Response Value = {attention_dict[columnValue]}, score = {attentionScore}')
#     print(f'Total Score: {attentionScore} | Normalized Score : {attentionScore/30}')
#     break     
    attention_scores.append(attentionScore/30)


attention_df['Attention Score'] = attention_scores

# len(attention_df[['randomID', 'Attention Score']])
# attention_df[['randomID', 'Attention Score']]
attention_df.to_excel(f'{prolificId_match_directory}attentionResponses.xlsx', index = False)
attention_df

Q163#1_1                   Q754#1_1  \
10             Strongly agree             Somewhat agree   
11             Strongly agree                      Agree   
12             Strongly agree             Strongly agree   
13             Strongly agree             Strongly agree   
21             Somewhat agree             Somewhat agree   
22             Strongly agree             Strongly agree   
23                      Agree                      Agree   
24                      Agree             Somewhat agree   
25             Somewhat agree  Neither agree or disagree   
26                      Agree                      Agree   
27             Strongly agree             Strongly agree   
28             Somewhat agree             Somewhat agree   
29             Strongly agree             Strongly agree   
30                      Agree                      Agree   
31             Strongly agree             Strongly agree   
32             Strongly agree                      Agree   
33             Strongly agree             Strongly agree   
34             Strongly agree  Neither agree or disagree   
35             Strongly agree                      Agree   
37             Strongly agree             Strongly agree   
38             Strongly agree                      Agree   
39             Strongly agree                      Agree   
40                        NaN                        NaN   
43                      Agree                      Agree   
44             Strongly agree             Strongly agree   
45  Neither agree or disagree                   Disagree   
46             Strongly agree             Strongly agree   
47             Strongly agree                      Agree   
48             Strongly agree             Strongly agree   
50                        NaN                        NaN   
51             Strongly agree             Strongly agree   
52             Strongly agree             Strongly agree   
53                      Agree                      Agree   
54  Neither agree or disagree  Neither agree or disagree   
55             Strongly agree             Somewhat agree   
56             Strongly agree             Strongly agree   
57                      Agree                      Agree   
58                      Agree                   Disagree   
59             Strongly agree             Strongly agree   
60                        NaN                        NaN   

                     Q761#1_1                   Q768#1_1  \
10             Somewhat agree                   Disagree   
11                      Agree                      Agree   
12             Strongly agree             Strongly agree   
13                      Agree  Neither agree or disagree   
21             Somewhat agree  Neither agree or disagree   
22             Strongly agree             Strongly agree   
23                      Agree             Strongly agree   
24  Neither agree or disagree  Neither agree or disagree   
25                   Disagree                   Disagree   
26                      Agree             Somewhat agree   
27             Strongly agree             Strongly agree   
28             Somewhat agree             Somewhat agree   
29             Strongly agree                      Agree   
30             Strongly agree          Somewhat disagree   
31             Strongly agree             Strongly agree   
32                      Agree                      Agree   
33             Strongly agree             Strongly agree   
34                      Agree                      Agree   
35             Strongly agree                      Agree   
37             Strongly agree             Strongly agree   
38             Strongly agree             Strongly agree   
39             Strongly agree             Strongly agree   
40                        NaN                        NaN   
43                      Agree                      Agree   
44             Strongly agree             Strongly agree   


In [13]:
prolificID_matches_df = prolificID_matches_df.merge(attention_df[['randomID', 'Attention Score']], 'inner', left_on = 'Participant', right_on = 'randomID').drop('randomID', axis=1)
prolificID_matches_df

Participant               Prolific ID Survey Duration  Attention Score
0         8006  602e9c46d37c26a6f0ec65ad         0:33:17        -0.100000
1         3260  5faa741b7e564e3971432a66         0:40:56         0.933333
2         7444  5f2ef7740f87702b99055397         0:33:07         0.933333
3         3518  6162226f379aa5c552808895         1:06:47        -0.166667
4         9214  601903a3842bcc20df15b296         0:40:17         0.200000
5         8417  5b8e8feeee5b0500010f16d0         0:27:14         0.733333
6         6110  6163fe9cd62993d57a578f0c         0:37:33         0.933333
7         1889  5f4894201c384e3607e91d57         0:45:49         0.066667
8         1357  6168174598124da7e933f76d         1:02:11        -0.400000
9         6953  5d40a12f4994c40001e4b80c         0:37:54        -0.333333
10        8586  5ddd2b7887c1fdc8d0375bd6         0:23:48         0.800000
11        5177  5bc1f0157870b900012dcb8a         0:27:04        -0.266667
12        5531  60ae642a030910da26e53328         0:45:51         0.733333
13        1941  60d090dc7a9fcd3436d75960         0:35:56         0.133333
14        4725  5fd24dc25191560a0dbe4c66         0:33:46        -0.333333
15        2039  60009e21e71d6d170204369d         0:29:03         1.000000
16        6686  615da788698e89290fe145e3         0:26:19        -0.333333
17        6253  5fd2159a30a24428526f9c69         0:44:45         0.266667
18        6566  6029795d5873ea8d25c1bfc9         0:32:47         0.766667
19        7440  5db9b910001ffa0188426dca         0:20:59        -0.333333
20        3147  60e6ca61f1fbd1edc317a63e         1:14:35         1.000000
21        2584  5f6214a59413141f7cb2508c         0:52:56         0.900000
22        2946  6124c375109ed10432fa2114         0:05:41       -10.000000
23        3499  5d595c1acf514a00177872ed         0:20:22        -0.333333
24        2987  5ed5c67a751f9814f8ffd344         0:24:22         0.833333
25        2088  6117bdb3ee7e7ca452e3199d         0:26:24        -0.366667
26        6618  616dc0a86b3ccbe7d735f712         0:33:43        -0.333333
27        1480  60f97c4d9f14764c11aefeea         0:43:59         0.033333
28        1525  60476cc0bb649624b9661b87         0:39:52         0.533333
29        8296  60e5a4a93d30a9f7f695a9b7         0:23:38        -8.800000
30        2903  61309cc42d77788e1605c645         0:18:40         0.966667
31        3947  5f346965f972db371654b614         0:30:42        -0.333333
32        3503  60f046be4df6e302b34526bf         0:31:02        -0.333333
33        7811  6031a3ce9c4b051348c05c4b         0:35:26        -0.133333
34        1499  5e15bb77e78021b9d45d8cf4         0:33:12        -0.266667
35        8845  616711b01b37853903a099e0         0:42:05         0.566667
36        2101  615e222d390962ba321edde6         0:45:14        -0.333333
37        3235  60ca98d006f5840320e8049e         0:26:31        -0.166667
38        2818  5e9c3ff1cf719d151612648f         1:06:55        -0.333333
39        1230  615ad6db4cd0092a52d09ea7         0:02:55       -10.000000

#### Score/video/participant

In [14]:
score_video_participant = attention_df
# score_video_participant

In [15]:
for index, row in attention_df.filter(items = contextResponseColumns(attention_df.columns), axis = 1).iterrows():
    attentionScore = 0
    for columnName, columnValue in row.items():
        try:
#             print(f'{columnName}: {context_dict[columnName]} Video, Response: {columnValue}, Response Value = {attention_dict[columnValue]}')
            if(context_dict[columnName] == 'CONTROL'):
                if (attention_dict[columnValue] > 3):
                    score_video_participant.at[index, columnName] = 1
                elif(attention_dict[columnValue] < 3):
                    score_video_participant.at[index, columnName] = -1
                else:
                    score_video_participant.at[index, columnName] = 0
            elif(context_dict[columnName] == 'FAIL'):
                if (attention_dict[columnValue] < 3):
                    score_video_participant.at[index, columnName] = 1
                elif(attention_dict[columnValue] > 3):
                    score_video_participant.at[index, columnName] = -1
                else:
                    score_video_participant.at[index, columnName] = 0
        except KeyError:
            continue
        except AttributeError:
            continue
score_video_participant

Q163#1_1 Q754#1_1 Q761#1_1 Q768#1_1 Q775#1_1 Q782#1_1 Q789#1_1 Q796#1_1  \
10        1        1        1       -1        1       -1        1        1   
11        1        1        1        1        1        1        1        1   
12        1        1        1        1        1        1        1        1   
13        1        1        1        0       -1       -1        1       -1   
21        1        1        1        0        1       -1        1        1   
22        1        1        1        1        1        1        1        1   
23        1        1        1        1        1        1        1       -1   
24        1        1        0        0       -1        1       -1       -1   
25        1        0       -1       -1        1        1        1       -1   
26        1        1        1        1        1        1        1        1   
27        1        1        1        1        1       -1        1        1   
28        1        1        1        1        0        1        1        1   
29        1        1        1        1        1        1        1        1   
30        1        1        1       -1       -1       -1        1        1   
31        1        1        1        1        1        1        1        1   
32        1        1        1        1        1        1        1        1   
33        1        1        1        1        1        1        1        1   
34        1        0        1        1        1        1        1        1   
35        1        1        1        1        1        1        1        1   
37        1        1        1        1        1        1        1        1   
38        1        1        1        1        1        1        1        1   
39        1        1        1        1        1        1        1        1   
40      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
43        1        1        1        1        1        1        1        1   
44        1        1        1        1        0        1        1        1   
45        0       -1        1       -1        1       -1        1        0   
46        1        1        1        1        1        1        1        1   
47        1        1        1        1        1        1        0        1   
48        1        1        1        1        1        1        1        1   
50      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
51        1        1        1        1        1        1        1        1   
52        1        1        1        0        1        1        1        1   
53        1        1        1        1        1        1        1        1   
54        0        0        1        0        0        1        0        0   
55        1        1        1       -1        1        1        1        1   
56        1        1        1        1        1        1        1        1   
57        1        1        1        1        1       -1       -1        1   
58        1       -1        0        1        1        1       -1       -1   
59        1        1        1        1        1        1        1        1   
60      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   

   Q803#1_1 Q810#1_1  ... Q901#1_1 Q908#1_1 Q915#1_1 Q922#1_1 Q929#1_1  \
10        0       -1  ...       -1        0        1        1        1   
11        1        1  ...        1       -1        1        1        1   
12        1        1  ...        1        1        1        1        1   
13        1        1  ...       -1        1       -1       -1        1   
21        1        1  ...        1       -1       -1       -1        1   
22        1        1  ...        1       -1        1        1        1   
23        1        1  ...        1        1        1        1        1   
24        0        1  ...        1        1       -1        1        1   
25        1        1  ...       -1       -1       -1       -1       -1   
26        1        1  ...       -1       -1       -1       -1   

In [16]:
score_video_participant.to_excel(f'{prolificId_match_directory}score_video_participant.xlsx', index = False)

#### 3. Update participant_log to include new survey user's information

In [17]:
print("Existing Survey Users:")
participant_log

Existing Survey Users:


Participant               Prolific ID Survey Duration       Type  \
0          8006  602e9c46d37c26a6f0ec65ad        00:33:17      Pilot   
1          3260  5faa741b7e564e3971432a66        00:40:56      Pilot   
2          7444  5f2ef7740f87702b99055397        00:33:07      Pilot   
3          3518  6162226f379aa5c552808895        01:06:47      Pilot   
4          9214  601903a3842bcc20df15b296        00:40:17  MainStudy   
5          8417  5b8e8feeee5b0500010f16d0        00:27:14  MainStudy   
6          6110  6163fe9cd62993d57a578f0c        00:37:33  MainStudy   
7          1889  5f4894201c384e3607e91d57        00:45:49  MainStudy   
8          1357  6168174598124da7e933f76d        01:02:11  MainStudy   
9          6953  5d40a12f4994c40001e4b80c        00:37:54  MainStudy   
10         8586  5ddd2b7887c1fdc8d0375bd6        00:23:48  MainStudy   
11         5177  5bc1f0157870b900012dcb8a        00:27:04  MainStudy   
12         5531  60ae642a030910da26e53328        00:45:51  MainStudy   
13         1941  60d090dc7a9fcd3436d75960        00:35:56  MainStudy   
14         4725  5fd24dc25191560a0dbe4c66        00:33:46  MainStudy   
15         2039  60009e21e71d6d170204369d        00:29:03  MainStudy   
16         6686  615da788698e89290fe145e3        00:26:19  MainStudy   
17         6253  5fd2159a30a24428526f9c69        00:44:45  MainStudy   
18         6566  6029795d5873ea8d25c1bfc9        00:32:47  MainStudy   
19         7440  5db9b910001ffa0188426dca        00:20:59  MainStudy   
20         3147  60e6ca61f1fbd1edc317a63e        01:14:35  MainStudy   
21         2584  5f6214a59413141f7cb2508c        00:52:56  MainStudy   
22         2946  6124c375109ed10432fa2114        00:05:41     Ignore   
23         3499  5d595c1acf514a00177872ed        00:20:22  MainStudy   
24         2987  5ed5c67a751f9814f8ffd344        00:24:22  MainStudy   
25         2088  6117bdb3ee7e7ca452e3199d        00:26:24  MainStudy   
26         6618  616dc0a86b3ccbe7d735f712        00:33:43  MainStudy   
27         1480  60f97c4d9f14764c11aefeea        00:43:59     Ignore   
28         1525  60476cc0bb649624b9661b87        00:39:52     Ignore   
29         8296  60e5a4a93d30a9f7f695a9b7        00:23:38     Ignore   
30         2903  61309cc42d77788e1605c645        00:18:40     Ignore   
31         3947  5f346965f972db371654b614        00:30:42  MainStudy   
32         3503  60f046be4df6e302b34526bf        00:31:02  MainStudy   
33         7811  6031a3ce9c4b051348c05c4b        00:35:26  MainStudy   
34         1499  5e15bb77e78021b9d45d8cf4        00:33:12  MainStudy   
35         8845  616711b01b37853903a099e0        00:42:05  MainStudy   
36         2101  615e222d390962ba321edde6        00:45:14  MainStudy   
37         3235  60ca98d006f5840320e8049e        00:26:31  MainStudy   
38         2818  5e9c3ff1cf719d151612648f        01:06:55  MainStudy   
39         1230  615ad6db4cd0092a52d09ea7        00:02:55     Ignore   

    Attention Score  Number of videos Video Good? Survey good?  \
0         -0.100000                31          No          Yes   
1          0.933333                34         Yes          Yes   
2          0.933333                32         Yes          Yes   
3         -0.166667                42     Neutral          Yes   
4          0.200000                30         Yes          Yes   
5          0.733333                30         Yes          Yes   
6          0.933333                33          No          Yes   
7          0.066667                33         Yes          Yes   
8         -0.400000                44         Yes          Yes   
9         -0.333333                30         Yes          Yes   
10         0.800000                30          30          Yes   
11        -0.266667                23     Neutral          Yes   
12         0.733333                30         Yes          Yes   
13         0.133333                33         Yes          Yes   
14        -0.333333                30         Yes          Ye

In [18]:
# Creating a new dataFrame equivalent to the participant_log document 
# Here, in this dataFrame we add all the new users who have taken the survey
newUsers = pd.DataFrame(columns=participant_log.columns)
newUsers

Empty DataFrame
Columns: [Participant, Prolific ID, Survey Duration, Type, Attention Score, Number of videos, Video Good?, Survey good?, Comments, Repeated Video #s, Pay?, Include as Study Data?]
Index: []

In [19]:
# Here, we iterate over all the Prolific ID & Participant ID matches
# Then we try to check if each of users in the matches already exist in the participant_log
# If they don't, then they are a newUser and they get appended to the newUsers dataframe
# The newUsers dataFrame is then append to the existing participant_log

for _, newUser in prolificID_matches_df.iterrows():
    newUserId = newUser['Prolific ID']
    found = False
    for _, user in participant_log.iterrows():
        userId = user['Prolific ID']
        if(newUserId == userId):
            found = True
    if (not found): # new user
        newUsers = newUsers.append(newUser, ignore_index=True)

print("The new survey users are as follows: ")
newUsers

The new survey users are as follows: 


Empty DataFrame
Columns: [Participant, Prolific ID, Survey Duration, Type, Attention Score, Number of videos, Video Good?, Survey good?, Comments, Repeated Video #s, Pay?, Include as Study Data?]
Index: []

In [20]:
# Here we concatenate the new users to the exisitng users who have taken the survey in the participant_log
participant_log = pd.concat([participant_log, newUsers], ignore_index = True)
participant_log

Participant               Prolific ID Survey Duration       Type  \
0         8006  602e9c46d37c26a6f0ec65ad        00:33:17      Pilot   
1         3260  5faa741b7e564e3971432a66        00:40:56      Pilot   
2         7444  5f2ef7740f87702b99055397        00:33:07      Pilot   
3         3518  6162226f379aa5c552808895        01:06:47      Pilot   
4         9214  601903a3842bcc20df15b296        00:40:17  MainStudy   
5         8417  5b8e8feeee5b0500010f16d0        00:27:14  MainStudy   
6         6110  6163fe9cd62993d57a578f0c        00:37:33  MainStudy   
7         1889  5f4894201c384e3607e91d57        00:45:49  MainStudy   
8         1357  6168174598124da7e933f76d        01:02:11  MainStudy   
9         6953  5d40a12f4994c40001e4b80c        00:37:54  MainStudy   
10        8586  5ddd2b7887c1fdc8d0375bd6        00:23:48  MainStudy   
11        5177  5bc1f0157870b900012dcb8a        00:27:04  MainStudy   
12        5531  60ae642a030910da26e53328        00:45:51  MainStudy   
13        1941  60d090dc7a9fcd3436d75960        00:35:56  MainStudy   
14        4725  5fd24dc25191560a0dbe4c66        00:33:46  MainStudy   
15        2039  60009e21e71d6d170204369d        00:29:03  MainStudy   
16        6686  615da788698e89290fe145e3        00:26:19  MainStudy   
17        6253  5fd2159a30a24428526f9c69        00:44:45  MainStudy   
18        6566  6029795d5873ea8d25c1bfc9        00:32:47  MainStudy   
19        7440  5db9b910001ffa0188426dca        00:20:59  MainStudy   
20        3147  60e6ca61f1fbd1edc317a63e        01:14:35  MainStudy   
21        2584  5f6214a59413141f7cb2508c        00:52:56  MainStudy   
22        2946  6124c375109ed10432fa2114        00:05:41     Ignore   
23        3499  5d595c1acf514a00177872ed        00:20:22  MainStudy   
24        2987  5ed5c67a751f9814f8ffd344        00:24:22  MainStudy   
25        2088  6117bdb3ee7e7ca452e3199d        00:26:24  MainStudy   
26        6618  616dc0a86b3ccbe7d735f712        00:33:43  MainStudy   
27        1480  60f97c4d9f14764c11aefeea        00:43:59     Ignore   
28        1525  60476cc0bb649624b9661b87        00:39:52     Ignore   
29        8296  60e5a4a93d30a9f7f695a9b7        00:23:38     Ignore   
30        2903  61309cc42d77788e1605c645        00:18:40     Ignore   
31        3947  5f346965f972db371654b614        00:30:42  MainStudy   
32        3503  60f046be4df6e302b34526bf        00:31:02  MainStudy   
33        7811  6031a3ce9c4b051348c05c4b        00:35:26  MainStudy   
34        1499  5e15bb77e78021b9d45d8cf4        00:33:12  MainStudy   
35        8845  616711b01b37853903a099e0        00:42:05  MainStudy   
36        2101  615e222d390962ba321edde6        00:45:14  MainStudy   
37        3235  60ca98d006f5840320e8049e        00:26:31  MainStudy   
38        2818  5e9c3ff1cf719d151612648f        01:06:55  MainStudy   
39        1230  615ad6db4cd0092a52d09ea7        00:02:55     Ignore   

    Attention Score Number of videos Video Good? Survey good?  \
0         -0.100000               31          No          Yes   
1          0.933333               34         Yes          Yes   
2          0.933333               32         Yes          Yes   
3         -0.166667               42     Neutral          Yes   
4          0.200000               30         Yes          Yes   
5          0.733333               30         Yes          Yes   
6          0.933333               33          No          Yes   
7          0.066667               33         Yes          Yes   
8         -0.400000               44         Yes          Yes   
9         -0.333333               30         Yes          Yes   
10         0.800000               30          30          Yes   
11        -0.266667               23     Neutral          Yes   
12         0.733333               30         Yes          Yes   
13         0.133333               33         Yes          Yes   
14        -0.333333               30         Yes          Yes   
15         1.000000               30         Yes   

#### 4. Calculate the number of responses provided in each survey

In [21]:
# Calculates the Number of Response Videos recorded by each survey participant

participantResponsePattern = r'QID\d+'

for participant in participant_log['Participant']:
    try:
        # print('Participant Directory is ->', f'{prolificId_match_directory}{participant}')
        participantResponses = os.listdir(f'{prolificId_match_directory}/{participant}')
        # print('Directory Files:\n', '\n'.join(participantResponses))
        numberOfParticipantResponses = len([response for response in participantResponses if re.match(participantResponsePattern, response)])
        # print('Number of Participant Responses = ', numberOfParticipantResponses)
    except FileNotFoundError:
        numberOfParticipantResponses = 0
    participant_log.loc[participant_log['Participant'] == participant, 'Number of videos'] = numberOfParticipantResponses

participant_log

Participant               Prolific ID Survey Duration       Type  \
0         8006  602e9c46d37c26a6f0ec65ad        00:33:17      Pilot   
1         3260  5faa741b7e564e3971432a66        00:40:56      Pilot   
2         7444  5f2ef7740f87702b99055397        00:33:07      Pilot   
3         3518  6162226f379aa5c552808895        01:06:47      Pilot   
4         9214  601903a3842bcc20df15b296        00:40:17  MainStudy   
5         8417  5b8e8feeee5b0500010f16d0        00:27:14  MainStudy   
6         6110  6163fe9cd62993d57a578f0c        00:37:33  MainStudy   
7         1889  5f4894201c384e3607e91d57        00:45:49  MainStudy   
8         1357  6168174598124da7e933f76d        01:02:11  MainStudy   
9         6953  5d40a12f4994c40001e4b80c        00:37:54  MainStudy   
10        8586  5ddd2b7887c1fdc8d0375bd6        00:23:48  MainStudy   
11        5177  5bc1f0157870b900012dcb8a        00:27:04  MainStudy   
12        5531  60ae642a030910da26e53328        00:45:51  MainStudy   
13        1941  60d090dc7a9fcd3436d75960        00:35:56  MainStudy   
14        4725  5fd24dc25191560a0dbe4c66        00:33:46  MainStudy   
15        2039  60009e21e71d6d170204369d        00:29:03  MainStudy   
16        6686  615da788698e89290fe145e3        00:26:19  MainStudy   
17        6253  5fd2159a30a24428526f9c69        00:44:45  MainStudy   
18        6566  6029795d5873ea8d25c1bfc9        00:32:47  MainStudy   
19        7440  5db9b910001ffa0188426dca        00:20:59  MainStudy   
20        3147  60e6ca61f1fbd1edc317a63e        01:14:35  MainStudy   
21        2584  5f6214a59413141f7cb2508c        00:52:56  MainStudy   
22        2946  6124c375109ed10432fa2114        00:05:41     Ignore   
23        3499  5d595c1acf514a00177872ed        00:20:22  MainStudy   
24        2987  5ed5c67a751f9814f8ffd344        00:24:22  MainStudy   
25        2088  6117bdb3ee7e7ca452e3199d        00:26:24  MainStudy   
26        6618  616dc0a86b3ccbe7d735f712        00:33:43  MainStudy   
27        1480  60f97c4d9f14764c11aefeea        00:43:59     Ignore   
28        1525  60476cc0bb649624b9661b87        00:39:52     Ignore   
29        8296  60e5a4a93d30a9f7f695a9b7        00:23:38     Ignore   
30        2903  61309cc42d77788e1605c645        00:18:40     Ignore   
31        3947  5f346965f972db371654b614        00:30:42  MainStudy   
32        3503  60f046be4df6e302b34526bf        00:31:02  MainStudy   
33        7811  6031a3ce9c4b051348c05c4b        00:35:26  MainStudy   
34        1499  5e15bb77e78021b9d45d8cf4        00:33:12  MainStudy   
35        8845  616711b01b37853903a099e0        00:42:05  MainStudy   
36        2101  615e222d390962ba321edde6        00:45:14  MainStudy   
37        3235  60ca98d006f5840320e8049e        00:26:31  MainStudy   
38        2818  5e9c3ff1cf719d151612648f        01:06:55  MainStudy   
39        1230  615ad6db4cd0092a52d09ea7        00:02:55     Ignore   

    Attention Score Number of videos Video Good? Survey good?  \
0         -0.100000               31          No          Yes   
1          0.933333               34         Yes          Yes   
2          0.933333               32         Yes          Yes   
3         -0.166667               42     Neutral          Yes   
4          0.200000               30         Yes          Yes   
5          0.733333               30         Yes          Yes   
6          0.933333               33          No          Yes   
7          0.066667               33         Yes          Yes   
8         -0.400000               44         Yes          Yes   
9         -0.333333               30         Yes          Yes   
10         0.800000               30          30          Yes   
11        -0.266667               23     Neutral          Yes   
12         0.733333               30         Yes          Yes   
13         0.133333               33         Yes          Yes   
14        -0.333333               30         Yes          Yes   
15         1.000000               30         Yes   

In [22]:
participant_log.to_csv(f'{prolificId_match_directory}participant_log.csv', index = False)